In [1]:
import pandas as pd
import numpy as np
import os, re
from typing import Union
from typing import Optional, Tuple
from functools import partial

In [2]:
df_fd = pd.read_csv("ignore/fd_summary.csv")
df_fd.head()

,metric,avg,std,cv,notes,prob_name,domain,planner,config
0,translate,0.339556,0.023291,0.068593,NaN,prob10:,zenotravel,fd,lmcut
1,scoping,2.395337,0.073150,0.030538,NaN,prob10:,zenotravel,fd,lmcut
2,translate_and_scope,2.734893,0.081340,0.029741,NaN,prob10:,zenotravel,fd,lmcut
3,plan_unscoped_time,37.723080,0.721374,0.019123,NaN,prob10:,zenotravel,fd,lmcut
4,plan_scoped_time,33.154035,0.405908,0.012243,NaN,prob10:,zenotravel,fd,lmcut


In [3]:
import itertools as it

In [4]:
def clean_col_name(s: str) -> str:
    return s.replace("_"," ").title()
def get_scoped_unscoped_names(s: str):
    return s + " (Scoped)", s + " (Unscoped)"

fd_colmap_manual = {
    "plan_unscoped_generated_nodes": "Eval. (Unscoped)",
    "plan_unscoped_node_expansions": "Exp. (Unscoped)",
    "plan_scoped_generated_nodes": "Eval. (Scoped)",
    "plan_scoped_node_expansions": "Exp. (Scoped)",
    "plan_unscoped_time": "Planning (Unscoped)",
    "plan_scoped_time": "Planning (Scoped)",
    "encoding_size": "Encoding Size",
    "total_unscoped_time": "Total (Unscoped)",
    "total_scoped_time": "Total (Scoped)"
}
paired_metrics =  ["Eval.", "Exp.", "Total", "Planning" ]
time_metrics = ["Total", "Planning", "Translate", "Scoping", "Translate And Scope"]
time_metrics_scoped_and_unscoped = list(it.chain(*[get_scoped_unscoped_names(m) for m in time_metrics]))
solo_metrics = ["Translate", "Scoping", "Translate And Scope"]

fd_colmap = {c: clean_col_name(c) for c in df_fd["metric"].unique()}
fd_colmap.update(fd_colmap_manual)
# df_fd = df_fd.rename(columns=fd_colmap)
# df_fd
df_fd["metric"] = df_fd["metric"].replace(fd_colmap)
config2keep = "ms"
df_fd = df_fd[df_fd["config"] == config2keep]
df_fd["prob_name"] = df_fd["prob_name"].str.slice(stop=-1)
df_fd

,metric,avg,std,cv,notes,prob_name,domain,planner,config
36,Translate,2.732207e-01,0.017759,0.065000,NaN,prob15,logistics,fd,ms
37,Scoping,6.790909e-01,0.032256,0.047499,NaN,prob15,logistics,fd,ms
38,Translate And Scope,9.523117e-01,0.049447,0.051923,NaN,prob15,logistics,fd,ms
39,Planning (Unscoped),7.371776e+01,2.922776,0.039648,NaN,prob15,logistics,fd,ms
40,Planning (Scoped),1.113938e+01,0.796540,0.071507,NaN,prob15,logistics,fd,ms
...,...,...,...,...,...,...,...,...,...
247,Eval. (Unscoped),>168064648,NaN,NaN,NaN,prob14,zenotravel,fd,ms
248,Exp. (Unscoped),>6462279,NaN,NaN,NaN,prob14,zenotravel,fd,ms
249,Eval. (Scoped),>130752144,NaN,NaN,NaN,prob14,zenotravel,fd,ms
250,Exp. (Scoped),>5392118,NaN,NaN,NaN,prob14,zenotravel,fd,ms


In [5]:
def parse_failure(df: pd.DataFrame, c: str) -> pd.DataFrame:
    df = df.copy()
    c_fail = "Failed"
    df[c_fail] = df[c].astype(str).str.startswith(">")
    df[c_fail] = df[c_fail] | df[c].isna()
    m = df[c_fail]
    df.loc[m, c] = df.loc[m, c].str.slice(1)
    df[c] = df[c].astype(float)
    return df

In [6]:
# def parse_timeouts(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
#     df = df.copy()
#     for c in cols:
#         c_timeout = c + " (Timed Out)"
#         df[c_timeout] = df[c].astype(str).str.startswith(">")
#         m = df[c_timeout]
#         df.loc[m, c] = df.loc[m, c].str.slice(1)
#         df[c] = df[c].astype(float)
#     return df

In [7]:
# df = parse_timeouts(df_fd, time_metrics_scoped_and_unscoped)
df = parse_failure(df_fd, "avg")
df

,metric,avg,std,cv,notes,prob_name,domain,planner,config,Failed
36,Translate,2.732207e-01,0.017759,0.065000,NaN,prob15,logistics,fd,ms,False
37,Scoping,6.790909e-01,0.032256,0.047499,NaN,prob15,logistics,fd,ms,False
38,Translate And Scope,9.523117e-01,0.049447,0.051923,NaN,prob15,logistics,fd,ms,False
39,Planning (Unscoped),7.371776e+01,2.922776,0.039648,NaN,prob15,logistics,fd,ms,False
40,Planning (Scoped),1.113938e+01,0.796540,0.071507,NaN,prob15,logistics,fd,ms,False
...,...,...,...,...,...,...,...,...,...,...
247,Eval. (Unscoped),1.680646e+08,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True
248,Exp. (Unscoped),6.462279e+06,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True
249,Eval. (Scoped),1.307521e+08,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True
250,Exp. (Scoped),5.392118e+06,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True


In [8]:
df[df["Failed"]]

,metric,avg,std,cv,notes,prob_name,domain,planner,config,Failed
171,Planning (Unscoped),8.133287e+02,4.175090,NaN,(100% mem-out),prob07,satellite,fd,ms,True
173,Total (Unscoped),8.381490e+02,18.288720,NaN,(100% mem-out),prob07,satellite,fd,ms,True
175,Eval. (Unscoped),3.623950e+08,NaN,NaN,(100% mem-out),prob07,satellite,fd,ms,True
176,Exp. (Unscoped),2.142340e+07,NaN,NaN,(100% mem-out),prob07,satellite,fd,ms,True
243,Planning (Unscoped),6.445603e+02,11.452420,0.017768,NaN,prob14,zenotravel,fd,ms,True
244,Planning (Scoped),6.613720e+02,17.038665,0.025763,NaN,prob14,zenotravel,fd,ms,True
245,Total (Unscoped),6.445603e+02,11.452420,NaN,NaN,prob14,zenotravel,fd,ms,True
246,Total (Scoped),6.761961e+02,17.093998,NaN,NaN,prob14,zenotravel,fd,ms,True
247,Eval. (Unscoped),1.680646e+08,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True
248,Exp. (Unscoped),6.462279e+06,NaN,NaN,NaN,prob14,zenotravel,fd,ms,True


In [9]:
def pair2comp_metric(s: str) -> str:
    return s + " (Comp)"

Make output dataframe shape

In [10]:
c_ind = ["domain", "prob_name"]
cols_p = list(df["metric"].unique()) + [pair2comp_metric(s) for s in paired_metrics]
df_p = pd.DataFrame(index=df[c_ind].drop_duplicates(keep="first").set_index(c_ind).index, columns=cols_p)
df_p

Translate Scoping Translate And Scope  \
domain     prob_name                                         
logistics  prob15          NaN     NaN                 NaN   
           prob20          NaN     NaN                 NaN   
           prob25          NaN     NaN                 NaN   
satellite  prob05          NaN     NaN                 NaN   
           prob06          NaN     NaN                 NaN   
           prob07          NaN     NaN                 NaN   
driverlog  prob15          NaN     NaN                 NaN   
           prob16          NaN     NaN                 NaN   
           prob17          NaN     NaN                 NaN   
zenotravel prob10          NaN     NaN                 NaN   
           prob12          NaN     NaN                 NaN   
           prob14          NaN     NaN                 NaN   

                     Planning (Unscoped) Planning (Scoped) Total (Unscoped)  \
domain     prob_name                                                          
logistics  prob15                    NaN               NaN              NaN   
           prob20                    NaN               NaN              NaN   
           prob25                    NaN               NaN              NaN   
satellite  prob05                    NaN               NaN              NaN   
           prob06                    NaN               NaN              NaN   
           prob07                    NaN               NaN              NaN   
driverlog  prob15                    NaN               NaN              NaN   
           prob16                    NaN               NaN              NaN   
           prob17                    NaN               NaN              NaN   
zenotravel prob10                    NaN               NaN              NaN   
           prob12                    NaN               NaN              NaN   
           prob14                    NaN               NaN              NaN   

                     Total (Scoped) Eval. (Unscoped) Exp. (Unscoped)  \
domain     prob_name                                                   
logistics  prob15               NaN              NaN             NaN   
           prob20               NaN              NaN             NaN   
           prob25               NaN              NaN             NaN   
satellite  prob05               NaN              NaN             NaN   
           prob06               NaN              NaN             NaN   
           prob07               NaN              NaN             NaN   
driverlog  prob15               NaN              NaN             NaN   
           prob16               NaN              NaN             NaN   
           prob17               NaN              NaN             NaN   
zenotravel prob10               NaN              NaN             NaN   
           prob12               NaN              NaN             NaN   
           prob14               NaN              NaN             NaN   

                     Eval. (Scoped) Exp. (Scoped) Encoding Size Eval. (Comp)  \
domain     prob_name                                                           
logistics  prob15               NaN           NaN           NaN          NaN   
           prob20               NaN           NaN           NaN          NaN   
           prob25               NaN           NaN           NaN          NaN   
satellite  prob05               NaN           NaN           NaN          NaN   
           prob06               NaN           NaN           NaN          NaN   
           prob07               NaN           NaN           NaN          NaN   
driverlog  prob15               NaN           NaN           NaN          NaN   
           prob16               NaN           NaN           NaN          NaN   
           prob17               NaN           NaN           NaN          NaN   
zenotravel prob10               NaN           NaN           NaN          NaN   
           prob12               NaN           NaN           NaN          NaN

Populate values into output dataframe

In [11]:
def get_fail_col(s: str) -> str:
    return s + " (Failed)"

def get_comp_col(s: str) -> str:
    return s + " (Comp)"

def get_sd_col(s: str) -> str:
    return s + " (STD)"

In [12]:
for i, r in df.iterrows():
    dom, prob, metric, v, sd, failed = r[["domain", "prob_name", "metric", "avg","std", "Failed"]]
    df_p.loc[(dom, prob), metric] = v
    c_fail = get_fail_col(metric)
    df_p.loc[(dom, prob), c_fail] = failed
    c_sd = get_sd_col(metric)
    df_p.loc[(dom, prob), c_sd] = sd

df_p

Translate    Scoping Translate And Scope  \
domain     prob_name                                            
logistics  prob15     0.273221   0.679091            0.952312   
           prob20     0.276705   0.677175            0.953881   
           prob25     0.274595   0.755359            1.029954   
satellite  prob05     0.312893   0.636884            0.949777   
           prob06     0.334338   0.499869            0.834207   
           prob07     0.388497   0.909504            1.298001   
driverlog  prob15      0.45551   3.641865            4.097375   
           prob16     0.694248   8.370862             9.06511   
           prob17     0.834222   9.529708            10.36393   
zenotravel prob10     0.341063   2.384334            2.725396   
           prob12     0.574093   7.330916             7.90501   
           prob14     0.985119  14.269181             15.2543   

                     Planning (Unscoped) Planning (Scoped) Total (Unscoped)  \
domain     prob_name                                                          
logistics  prob15               73.71776          11.13938         73.99098   
           prob20               7.789221          8.551329         8.065926   
           prob25               147.9828          21.51109         148.2574   
satellite  prob05               3.873572          4.512795         4.186465   
           prob06               1.041538           1.39082         1.365033   
           prob07               813.3287          203.1748          838.149   
driverlog  prob15               12.39936          9.409023         12.85487   
           prob16               7.580889          4.517639         8.275136   
           prob17               154.0327          50.24538          154.867   
zenotravel prob10               22.13183          16.55866         22.47289   
           prob12               125.3582          72.29238         125.9323   
           prob14             644.560291        661.371979       644.560291   

                     Total (Scoped) Eval. (Unscoped) Exp. (Unscoped)  \
domain     prob_name                                                   
logistics  prob15          12.09169      140607200.0       5951997.0   
           prob20           9.50521        6941424.0        289584.0   
           prob25          22.54104      265871100.0      11437240.0   
satellite  prob05          5.462572           7001.0           114.0   
           prob06          2.225911           1084.0            21.0   
           prob07          204.4728      362395000.0      21423400.0   
driverlog  prob15           13.5064        8796110.0        527636.0   
           prob16         13.582749         925264.0         38681.0   
           prob17          60.60931      198623900.0       7768684.0   
zenotravel prob10          19.28406       37782140.0       1466718.0   
           prob12          80.19739      231592500.0       5306442.0   
           prob14        676.196084      168064648.0       6462279.0   

                     Eval. (Scoped)  ... Eval. (Unscoped) (Failed)  \
domain     prob_name                 ...                             
logistics  prob15        11646320.0  ...                     False   
           prob20         1524997.0  ...                     False   
           prob25        35198000.0  ...                     False   
satellite  prob05            3950.0  ...                     False   
           prob06             684.0  ...                     False   
           prob07       679734000.0  ...                      True   
driverlog  prob15         7289831.0  ...                     False   
           prob16          577118.0  ...                     False   
           prob17        88705990.0  ...                     False   
zenotravel prob10        25677010.0  ...                     False   
           prob12       140117800.0  ...                     False   
           prob14       130752144.0  ...                      True   

             

Add size data

In [13]:
def pivot_df_size(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"problem":"prob_name"})
    cols_index = ["domain", "prob_name"]
    df_2 = df[cols_index].drop_duplicates(keep="first")
    df_2 = df_2.set_index(cols_index)
    for i, r in df.iterrows():
        c_operators = "Operators (" + r["scoped"].title() + ")"
        dom, prob, ops = r[["domain", "prob_name", "operators"]]
        df_2.loc[(dom, prob), c_operators] = ops
    df_2[["Operators (Scoped)", "Operators (Unscoped)"]] = df_2[["Operators (Scoped)", "Operators (Unscoped)"]].astype(int)
    return df_2

In [14]:
df_size = pivot_df_size(pd.read_csv("domain_sizes.csv"))
df_size

Operators (Scoped)  Operators (Unscoped)
domain     prob_name                                          
driverlog  prob17                   3770                  6170
           prob15                   2112                  2592
           prob16                   3540                  4890
zenotravel prob10                   1095                  1155
           prob12                   3159                  3375
           prob14                   6200                  6700
satellite  prob06                    362                   582
           prob07                    587                   983
           prob05                    339                   609
logistics  prob20                    250                   650
           prob15                    250                   650
           prob25                    290                   650

In [15]:
df_p = df_p.join(df_size, how="outer")
df_p

Translate    Scoping Translate And Scope  \
domain     prob_name                                            
driverlog  prob15      0.45551   3.641865            4.097375   
           prob16     0.694248   8.370862             9.06511   
           prob17     0.834222   9.529708            10.36393   
logistics  prob15     0.273221   0.679091            0.952312   
           prob20     0.276705   0.677175            0.953881   
           prob25     0.274595   0.755359            1.029954   
satellite  prob05     0.312893   0.636884            0.949777   
           prob06     0.334338   0.499869            0.834207   
           prob07     0.388497   0.909504            1.298001   
zenotravel prob10     0.341063   2.384334            2.725396   
           prob12     0.574093   7.330916             7.90501   
           prob14     0.985119  14.269181             15.2543   

                     Planning (Unscoped) Planning (Scoped) Total (Unscoped)  \
domain     prob_name                                                          
driverlog  prob15               12.39936          9.409023         12.85487   
           prob16               7.580889          4.517639         8.275136   
           prob17               154.0327          50.24538          154.867   
logistics  prob15               73.71776          11.13938         73.99098   
           prob20               7.789221          8.551329         8.065926   
           prob25               147.9828          21.51109         148.2574   
satellite  prob05               3.873572          4.512795         4.186465   
           prob06               1.041538           1.39082         1.365033   
           prob07               813.3287          203.1748          838.149   
zenotravel prob10               22.13183          16.55866         22.47289   
           prob12               125.3582          72.29238         125.9323   
           prob14             644.560291        661.371979       644.560291   

                     Total (Scoped) Eval. (Unscoped) Exp. (Unscoped)  \
domain     prob_name                                                   
driverlog  prob15           13.5064        8796110.0        527636.0   
           prob16         13.582749         925264.0         38681.0   
           prob17          60.60931      198623900.0       7768684.0   
logistics  prob15          12.09169      140607200.0       5951997.0   
           prob20           9.50521        6941424.0        289584.0   
           prob25          22.54104      265871100.0      11437240.0   
satellite  prob05          5.462572           7001.0           114.0   
           prob06          2.225911           1084.0            21.0   
           prob07          204.4728      362395000.0      21423400.0   
zenotravel prob10          19.28406       37782140.0       1466718.0   
           prob12          80.19739      231592500.0       5306442.0   
           prob14        676.196084      168064648.0       6462279.0   

                     Eval. (Scoped)  ... Exp. (Unscoped) (Failed)  \
domain     prob_name                 ...                            
driverlog  prob15         7289831.0  ...                    False   
           prob16          577118.0  ...                    False   
           prob17        88705990.0  ...                    False   
logistics  prob15        11646320.0  ...                    False   
           prob20         1524997.0  ...                    False   
           prob25        35198000.0  ...                    False   
satellite  prob05            3950.0  ...                    False   
           prob06             684.0  ...                    False   
           prob07       679734000.0  ...                     True   
zenotravel prob10        25677010.0  ...                    False   
           prob12       140117800.0  ...                    False   
           prob14       130752144.0  ...                     True   

                     Exp. (

In [16]:
paired_metrics.append("Operators")

Calculate which did better - scoped, unscoped, or neither - for relevant metrics

Lexicographic ranking based on failure, then value (lower is better)

TODO? failure should be for domain and scoped/unscoped, not per-metric

In [17]:
for c in paired_metrics:
    c_comp = get_comp_col(c)
    c_scoped, c_unscoped = get_scoped_unscoped_names(c)

    # Handle comparisons based on failures
    if c == "Operators":
        m_fail_scoped = pd.Series(data=False, index=df_p.index)
        m_fail_unscoped = pd.Series(data=False, index=df_p.index)
    else:
        m_fail_scoped = df_p[get_fail_col(c_scoped)]
        m_fail_unscoped = df_p[get_fail_col(c_unscoped)]
        
        df_p.loc[m_fail_scoped & m_fail_unscoped, c_comp] = "tie"
        df_p.loc[m_fail_scoped & ~m_fail_unscoped, c_comp] = "unscoped"
        df_p.loc[~m_fail_scoped & m_fail_unscoped, c_comp] = "scoped"

    # Handle other comparisons
    m_no_fail = ~(m_fail_scoped | m_fail_unscoped)
    m_scope_smaller = df_p[c_scoped] < df_p[c_unscoped]
    m_even = df_p[c_scoped] == df_p[c_unscoped]
    m_unscope_smaller = df_p[c_scoped] > df_p[c_unscoped]
    df_p.loc[m_scope_smaller & m_no_fail, c_comp] = "scoped"
    df_p.loc[m_even & m_no_fail, c_comp] = "tie"
    df_p.loc[m_unscope_smaller & m_no_fail, c_comp] = "unscoped"
df_p



Translate    Scoping Translate And Scope  \
domain     prob_name                                            
driverlog  prob15      0.45551   3.641865            4.097375   
           prob16     0.694248   8.370862             9.06511   
           prob17     0.834222   9.529708            10.36393   
logistics  prob15     0.273221   0.679091            0.952312   
           prob20     0.276705   0.677175            0.953881   
           prob25     0.274595   0.755359            1.029954   
satellite  prob05     0.312893   0.636884            0.949777   
           prob06     0.334338   0.499869            0.834207   
           prob07     0.388497   0.909504            1.298001   
zenotravel prob10     0.341063   2.384334            2.725396   
           prob12     0.574093   7.330916             7.90501   
           prob14     0.985119  14.269181             15.2543   

                     Planning (Unscoped) Planning (Scoped) Total (Unscoped)  \
domain     prob_name                                                          
driverlog  prob15               12.39936          9.409023         12.85487   
           prob16               7.580889          4.517639         8.275136   
           prob17               154.0327          50.24538          154.867   
logistics  prob15               73.71776          11.13938         73.99098   
           prob20               7.789221          8.551329         8.065926   
           prob25               147.9828          21.51109         148.2574   
satellite  prob05               3.873572          4.512795         4.186465   
           prob06               1.041538           1.39082         1.365033   
           prob07               813.3287          203.1748          838.149   
zenotravel prob10               22.13183          16.55866         22.47289   
           prob12               125.3582          72.29238         125.9323   
           prob14             644.560291        661.371979       644.560291   

                     Total (Scoped) Eval. (Unscoped) Exp. (Unscoped)  \
domain     prob_name                                                   
driverlog  prob15           13.5064        8796110.0        527636.0   
           prob16         13.582749         925264.0         38681.0   
           prob17          60.60931      198623900.0       7768684.0   
logistics  prob15          12.09169      140607200.0       5951997.0   
           prob20           9.50521        6941424.0        289584.0   
           prob25          22.54104      265871100.0      11437240.0   
satellite  prob05          5.462572           7001.0           114.0   
           prob06          2.225911           1084.0            21.0   
           prob07          204.4728      362395000.0      21423400.0   
zenotravel prob10          19.28406       37782140.0       1466718.0   
           prob12          80.19739      231592500.0       5306442.0   
           prob14        676.196084      168064648.0       6462279.0   

                     Eval. (Scoped)  ... Exp. (Unscoped) (STD)  \
domain     prob_name                 ...                         
driverlog  prob15         7289831.0  ...                   0.0   
           prob16          577118.0  ...                   0.0   
           prob17        88705990.0  ...                   0.0   
logistics  prob15        11646320.0  ...                   0.0   
           prob20         1524997.0  ...                   0.0   
           prob25        35198000.0  ...                   0.0   
satellite  prob05            3950.0  ...                   0.0   
           prob06             684.0  ...                   0.0   
           prob07       679734000.0  ...                   NaN   
zenotravel prob10        25677010.0  ...                   0.0   
           prob12       140117800.0  ...                   0.0   
           prob14       130752144.0  ...                   NaN   

                     Eval. (Scoped) (Failed) Eval. (Scoped) (STD)  \


Created df holding final oclumn names and formatted values (TODO)

In [18]:
def format_float(a: float, n_dec: Optional[int]) -> str:
    a = round(a, n_dec)
    if n_dec is None:
        a = int(a)
        a = f"{a:,}"
    else:
        a = f"{a:,.1f}"
    return a

TODO replace nans with vals from new experiments.

TODO add +- for the times

In [19]:
df_p["Translate And Scope"]

domain      prob_name
driverlog   prob15       4.097375
            prob16        9.06511
            prob17       10.36393
logistics   prob15       0.952312
            prob20       0.953881
            prob25       1.029954
satellite   prob05       0.949777
            prob06       0.834207
            prob07       1.298001
zenotravel  prob10       2.725396
            prob12        7.90501
            prob14        15.2543
Name: Translate And Scope, dtype: object

In [20]:
cols_out = list(df["metric"].unique())
df_out = pd.DataFrame(index=df_p.index, columns=cols_out, dtype=str)
# df_out[solo_metrics] = df_p[solo_metrics]
for metric in solo_metrics:
    if metric in time_metrics:
        n_dec = 1
    else:
        n_dec = None

    for i in df_p.index:
        s_metric = format_float(df_p.loc[i, metric], n_dec)

        # Add sd for timed metrics
        if metric in time_metrics:
            s_sd = df_p.loc[i, get_sd_col(metric)]
            if not np.isnan(s_sd):
                s_sd = format_float(s_sd, n_dec=n_dec)
                s_metric = s_metric + " \pm " + s_sd

        df_out.loc[i, metric] = "$" + s_metric + "$"

# df_out
for metric in paired_metrics:
    c_scoped, c_unscoped = get_scoped_unscoped_names(metric)
    c_comp = get_comp_col(metric)
    if metric in time_metrics:
        n_dec = 1
    else:
        n_dec = None
    for i in df_p.index:
        # Get content of strings
        s_scoped = format_float(df_p.loc[i, c_scoped], n_dec=n_dec)
        s_unscoped = format_float(df_p.loc[i, c_unscoped], n_dec=n_dec)
        # Add sd for timed metrics
        if metric in time_metrics:
            sd_scoped = df_p.loc[i, get_sd_col(c_scoped)]
            if not np.isnan(sd_scoped):
                sd_scoped = format_float(sd_scoped, n_dec=n_dec)
                s_scoped = s_scoped + " \pm " + sd_scoped

            sd_unscoped = df_p.loc[i, get_sd_col(c_unscoped)]
            if not np.isnan(sd_unscoped):
                sd_unscoped = format_float(sd_unscoped, n_dec=n_dec)
                s_unscoped = s_unscoped + " \pm " + sd_unscoped

        # Put > before failed runs
        if metric != "Operators":
            if df_p.loc[i, get_fail_col(c_scoped)]:
                s_scoped = "\gt " + s_scoped
            if df_p.loc[i, get_fail_col(c_unscoped)]:
                s_unscoped = "\gt " + s_unscoped
        

        # Bold the winner
        match df_p.loc[i, c_comp]:
            case "scoped":
                s_scoped = "\\mathbf{" + s_scoped + "}"
            case "unscoped":
                s_unscoped = "\\mathbf{" + s_unscoped + "}"
            case "tie":
                pass
            case other:
                raise ValueError(f"Unknown value {other}")

        # Cash money
        s_scoped = "$" + s_scoped + "$"
        s_unscoped = "$" + s_unscoped + "$"

        df_out.loc[i, c_scoped] = s_scoped
        df_out.loc[i, c_unscoped] = s_unscoped


df_out

Translate         Scoping Translate And Scope  \
domain     prob_name                                                      
driverlog  prob15     $0.5 \pm 0.0$   $3.6 \pm 0.2$       $4.1 \pm 0.2$   
           prob16     $0.7 \pm 0.0$   $8.4 \pm 0.4$       $9.1 \pm 0.4$   
           prob17     $0.8 \pm 0.0$   $9.5 \pm 0.3$      $10.4 \pm 0.3$   
logistics  prob15     $0.3 \pm 0.0$   $0.7 \pm 0.0$       $1.0 \pm 0.0$   
           prob20     $0.3 \pm 0.0$   $0.7 \pm 0.0$       $1.0 \pm 0.1$   
           prob25     $0.3 \pm 0.0$   $0.8 \pm 0.0$       $1.0 \pm 0.1$   
satellite  prob05     $0.3 \pm 0.1$   $0.6 \pm 0.1$       $0.9 \pm 0.1$   
           prob06     $0.3 \pm 0.1$   $0.5 \pm 0.1$       $0.8 \pm 0.0$   
           prob07     $0.4 \pm 0.1$   $0.9 \pm 0.1$       $1.3 \pm 0.0$   
zenotravel prob10     $0.3 \pm 0.0$   $2.4 \pm 0.0$       $2.7 \pm 0.0$   
           prob12     $0.6 \pm 0.0$   $7.3 \pm 0.1$       $7.9 \pm 0.2$   
           prob14     $1.0 \pm 0.0$  $14.3 \pm 0.3$      $15.3 \pm 0.3$   

                         Planning (Unscoped)         Planning (Scoped)  \
domain     prob_name                                                     
driverlog  prob15             $12.4 \pm 0.5$    $\mathbf{9.4 \pm 0.4}$   
           prob16              $7.6 \pm 0.2$    $\mathbf{4.5 \pm 0.2}$   
           prob17            $154.0 \pm 4.8$   $\mathbf{50.2 \pm 1.5}$   
logistics  prob15             $73.7 \pm 2.9$   $\mathbf{11.1 \pm 0.8}$   
           prob20     $\mathbf{7.8 \pm 0.4}$             $8.6 \pm 0.5$   
           prob25            $148.0 \pm 5.2$   $\mathbf{21.5 \pm 1.1}$   
satellite  prob05     $\mathbf{3.9 \pm 0.2}$             $4.5 \pm 0.2$   
           prob06     $\mathbf{1.0 \pm 0.1}$             $1.4 \pm 0.1$   
           prob07        $\gt 813.3 \pm 4.2$  $\mathbf{203.2 \pm 5.6}$   
zenotravel prob10             $22.1 \pm 0.4$   $\mathbf{16.6 \pm 0.3}$   
           prob12            $125.4 \pm 4.9$   $\mathbf{72.3 \pm 3.6}$   
           prob14       $\gt 644.6 \pm 11.5$      $\gt 661.4 \pm 17.0$   

                             Total (Unscoped)            Total (Scoped)  \
domain     prob_name                                                      
driverlog  prob15     $\mathbf{12.9 \pm 0.5}$            $13.5 \pm 0.5$   
           prob16      $\mathbf{8.3 \pm 0.3}$            $13.6 \pm 0.6$   
           prob17             $154.9 \pm 4.9$   $\mathbf{60.6 \pm 1.8}$   
logistics  prob15              $74.0 \pm 2.9$   $\mathbf{12.1 \pm 0.8}$   
           prob20      $\mathbf{8.1 \pm 0.4}$             $9.5 \pm 0.6$   
           prob25             $148.3 \pm 5.3$   $\mathbf{22.5 \pm 1.1}$   
satellite  prob05      $\mathbf{4.2 \pm 0.2}$             $5.5 \pm 0.3$   
           prob06      $\mathbf{1.4 \pm 0.1}$             $2.2 \pm 0.1$   
           prob07        $\gt 838.1 \pm 18.3$  $\mathbf{204.5 \pm 5.7}$   
zenotravel prob10              $22.5 \pm 0.4$   $\mathbf{19.3 \pm 0.4}$   
           prob12             $125.9 \pm 5.0$   $\mathbf{80.2 \pm 3.7}$   
           prob14        $\gt 644.6 \pm 11.5$      $\gt 676.2 \pm 17.1$   

                       Eval. (Unscoped)   Exp. (Unscoped)  \
domain     prob_name                                        
driverlog  prob15           $8,796,110$         $527,636$   
           prob16             $925,264$          $38,681$   
           prob17         $198,623,900$       $7,768,684$   
logistics  prob15         $140,607,200$       $5,951,997$   
           prob20           $6,941,424$         $289,584$   
           prob25         $265,871,100$      $11,437,240$   
satellite  prob05               $7,001$             $114$   
           prob06               $1,084$              $21$   
           prob07     $\gt 362,395,000$  $\gt 21,423,400$   
zenotravel prob10          $37,782,140$       $1,466,718$   
           prob12         $231,592,500$       $5,306,442$   
           prob14     $\gt 168,064,648$   $\gt 6,462,279$   

                              Ev

In [21]:
def df2tex(df: pd.DataFrame, label: str, caption: str) -> str:
    
    # Header
    # rows = ["\\begin{table*}[t]","\\resizebox{\\textwidth}{!}{%","\\begin{tabular}{" + "l"*len(df.columns) + "}"]
    # rows.append("\\toprule")
    # rows.append(" & ".join(df.columns) + "\\\\ \\midrule")
    rows = [r"""\begin{table*}[t]
\resizebox{\textwidth}{!}{%
\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
\textbf{Problem} & \multicolumn{2}{c}{\textbf{Operators}} & \multicolumn{2}{c}{\textbf{Expansions}} & \multicolumn{2}{c}{\textbf{Evaluations}} & \textbf{Translate} & \textbf{Scoping} & \multicolumn{2}{c}{\textbf{Planning Time (s)}} & \multicolumn{2}{c}{\textbf{Total Time (s)}}\\
  & Unscoped & Scoped & Unscoped & Scoped & Unscoped & Scoped &  &  & Unscoped & Scoped & Unscoped & Scoped\\
\midrule"""]

    # Rows
    for ind, r in df.iterrows():
        rows.append(" & ".join(list(r.values)) + "\\\\")
    rows.append("\\bottomrule")
    rows.append("\\end{tabular}")
    rows.append("}")
    rows += ["\\caption{" + caption + "}",
    "\\label{" + label + "}", 
    "\\end{table*}"]
    return "\n".join(rows)

In [22]:
df_out_2 = df_out.copy().reset_index(drop=False)
for i, r in df_out_2.iterrows():
    df_out_2.loc[i, "Problem"] = (df_out_2.loc[i, "domain"] + " " +  df_out_2.loc[i, "prob_name"].split("prob")[1]).title()

cols_out = ["Problem", "Operators (Unscoped)", "Operators (Scoped)", "Exp. (Unscoped)", "Exp. (Scoped)", "Eval. (Unscoped)", "Eval. (Scoped)", "Translate", "Scoping", "Planning (Unscoped)", "Planning (Scoped)", "Total (Unscoped)", "Total (Scoped)"]

df_out_2 = df_out_2[cols_out]
df_out_2.head(2)

,Problem,Operators (Unscoped),Operators (Scoped),Exp. (Unscoped),Exp. (Scoped),Eval. (Unscoped),Eval. (Scoped),Translate,Scoping,Planning (Unscoped),Planning (Scoped),Total (Unscoped),Total (Scoped)
0,Driverlog 15,"$2,592$","$\mathbf{2,112}$","$527,636$","$\mathbf{460,244}$","$8,796,110$","$\mathbf{7,289,831}$",$0.5 \pm 0.0$,$3.6 \pm 0.2$,$12.4 \pm 0.5$,$\mathbf{9.4 \pm 0.4}$,$\mathbf{12.9 \pm 0.5}$,$13.5 \pm 0.5$
1,Driverlog 16,"$4,890$","$\mathbf{3,540}$","$38,681$","$\mathbf{29,618}$","$925,264$","$\mathbf{577,118}$",$0.7 \pm 0.0$,$8.4 \pm 0.4$,$7.6 \pm 0.2$,$\mathbf{4.5 \pm 0.2}$,$\mathbf{8.3 \pm 0.3}$,$13.6 \pm 0.6$


In [23]:

s_out = df2tex(df_out_2, label="table:fd_experiments_ms", caption="Results for our Fast Downward experiments using the Merge and Shrink heuristic.")
with open("tmp.tex", "w") as f:
    f.write(s_out)

Only include lmcut in the main paper, remove ms. That can go in the appendix.
Remove Config column
Add some sort of size column(s)
Rename "Generated Nodes (...)" -> "Eval. (...)"
Rename "Node Expansions (...)" -> "Exp. (...)"
Remove decimal point and ± std from all Eval./Exp. columns
If it's still too wide, use a 2-line header
Column order:
cols = ["Problem", "Operators (Scoped)", "Operators (Unscoped)", "Exp. (Scoped)", "Exp. (Unscoped)", "Eval. (Scoped)", "Eval. (Unscoped)", "Translate", "Scoping", "Planning (Scoped)", "Planning (Unscoped)", "Total (Scoped)", "Total (Unscoped)"]